In [1]:
%load_ext pycozo.ipyext_direct

In [2]:
:create embedding_models {
    name: String,
    dimensions: Int,
    requires_instruction: Bool default false,
    provider: String? default null,
}

,status
0,OK


In [3]:
?[name, dimensions, requires_instruction, provider] <- [
    ["e5-base-v2", 768, true, "embaas.io"],
    ["e5-large-v2", 1024, true, "embaas.io"],
]

:put embedding_models {
    name,
    dimensions,
    requires_instruction,
    provider
}

,status
0,OK


In [4]:
:create lm_cache {
    chatml: Json,
    model: String,
    params: Json default {},
    expires_at: Float? default null,
    =>
    response: Json,
    embedding: <F32; 768>
}

,status
0,OK


In [5]:
::hnsw create lm_cache:embedding_space {
    dim: 768,
    m: 50,
    dtype: F32,
    fields: [embedding],
    distance: Cosine,
    ef_construction: 20,
    extend_candidates: false,
    keep_pruned_connections: false,
}

,status
0,OK


In [6]:
:create conceptnet {
    concept_id: Uuid,
    concept: String,
    description: String default "",
    embedding: <F32; 768>,
}


,status
0,OK


In [7]:
::hnsw create conceptnet:embedding_space {
    dim: 768,
    m: 50,
    dtype: F32,
    fields: embedding,
    distance: Cosine,
    ef_construction: 20,
    extend_candidates: false,
    keep_pruned_connections: false,
}

,status
0,OK


In [8]:
::lsh create conceptnet:concept {
    extractor: concept,
    tokenizer: Simple,
    n_perm: 200,
    target_threshold: 0.8,
    n_gram: 3,
    false_positive_weight: 1.0,
    false_negative_weight: 1.0,
}

,status
0,OK


In [9]:
:create entities {
    entity_id: Uuid,
    name: String,
    character_id: Uuid,
    description: String default "",
}


,status
0,OK


In [10]:
:create relations {
    subject: Uuid,
    subject_type: String,
    relation: String,
    object: Uuid,
    object_type: String,
    =>
    weight: Float default 1.0,
}


,status
0,OK


In [11]:
::lsh create entities:name {
    extractor: name,
    tokenizer: Simple,
    n_perm: 200,
    target_threshold: 0.8,
    n_gram: 5,
    false_positive_weight: 1.0,
    false_negative_weight: 1.0,
}

,status
0,OK


In [12]:
:create characters {
    character_id: Uuid,
    name: String,
    is_human: Bool default false,
    updated_at: Validity default [floor(now()), true],
    =>
    about: String default "",
    metadata: Json default {},
    model: String? default null,
}

,status
0,OK


In [13]:
::lsh create characters:name {
    extractor: name,
    tokenizer: Simple,
    n_perm: 50,
    target_threshold: 0.8,
    n_gram: 3,
    false_positive_weight: 1.0,
    false_negative_weight: 1.0,
}

,status
0,OK


In [14]:
:create sessions {
    session_id: Uuid,
    updated_at: Validity default [floor(now()), true],
    =>
    situation: String,
    summary: String? default null,
    metadata: Json default {},
}

,status
0,OK


In [15]:
::fts create sessions:summary {
    extractor: summary,
    extract_filter: !is_null(summary),
    tokenizer: Simple,
    filters: [AsciiFolding, AlphaNumOnly, Lowercase, Stemmer('english'), Stopwords('en')],
}

,status
0,OK


In [16]:
:create session_characters {
    session_id: Uuid,
    character_id: Uuid,
}

,status
0,OK


In [17]:
:create entries {
    session_id: Uuid,
    role: String,
    name: String? default null,
    updated_at: Validity default [floor(now()), true],
    =>
    content: String,
    character_id: Uuid? default null,
    sentiment: Float? default null,
}

,status
0,OK


In [18]:
:create episodes {
    episode_id: Uuid,
    character_id: Uuid,
    summary: String,
    last_accessed_at: Validity default [floor(now()), true],
    =>
    parent_episode: Uuid? default null,
    duration: Float default 0,
    embedding: <F32; 768>,
}

,status
0,OK


In [19]:
::hnsw create episodes:embedding_space {
    dim: 768,
    m: 50,
    dtype: F32,
    fields: [embedding],
    distance: Cosine,
    filter: !is_null(embedding),
    ef_construction: 20,
    extend_candidates: false,
    keep_pruned_connections: false,
}

,status
0,OK


In [20]:
::fts create episodes:summary {
    extractor: summary,
    extract_filter: !is_null(summary),
    tokenizer: Simple,
    filters: [AsciiFolding, AlphaNumOnly, Lowercase, Stemmer('english'), Stopwords('en')],
}

,status
0,OK


In [21]:
:create users {
    email: String,
    character_id: Uuid,
    =>
    assistant_id: Uuid,
}


,status
0,OK


In [22]:
:create beliefs {
    belief_id: Uuid,
    character_id: Uuid,
    belief: String,
    last_accessed_at: Validity default [floor(now()), true],
    =>
    details: String default "",
    parent_belief_id: Uuid? default null,
    valence: Float default 0,
    aspects: [(String, Float, String, String)] default [],
    belief_embedding: <F32; 768>,
    details_embedding: <F32; 768>,
}

,status
0,OK


In [23]:
::hnsw create beliefs:embedding_space {
    dim: 768,
    m: 50,
    dtype: F32,
    fields: [belief_embedding, details_embedding],
    distance: Cosine,
    ef_construction: 20,
    extend_candidates: false,
    keep_pruned_connections: false,
}

,status
0,OK


In [24]:
::fts create beliefs:summary {
    extractor: belief ++ " " ++ details,
    tokenizer: Simple,
    filters: [AsciiFolding, AlphaNumOnly, Lowercase, Stemmer('english'), Stopwords('en')],
}

,status
0,OK
